## **Second project:** Analysis of the evolution of Mass Shootings in the US

Authors: **Raquel Jolis Carné and Martina Massana Massip**

**Uploading the folder *visualization_data*** will ensure all necessary datasets are loaded to properly treat them in this notebook.

## **Data Cleaning with OpenRefine**

The only document we done a thorough data cleaning for is *MassShootings.csv*, which we have conducted the following procedure.  

1. Changing the type of **numerical data columns from *strings* to *integers*.** As well as setting the ***Incident Date*** column as a ***timestamp.***
2. Combining the columns ***State, City or County* and *Address* into a single *Complete_Address*** with the three fields.
3. Extracting **OpenStreetMap coordinates** for the incidents' complete addresses into a new *Coordinates* column.  
4. **Erasing rows** where **coordinates** were **not found.**
5. **Separating the *Coordinates*** values into two columns: ***Longitude* and *Latitude.***
6. Extracting ***FIPS* codes and *Population*** for each state by **fetching information from wikidata** using a Reconciling facet in OpenRefine.
7. Adding a new column ***Region*** with categories: *Midwest*, *Northeast*, *Southeast*, *Soutwest* and *West*.

Additional transformations to answer the concrete questions have been specified in the pertinent exercices' creation of *Pandas* dataframes by joining multiple datasets and selecting relevant columns.

In [51]:
!pip install -q altair==5.4.1

In [52]:
import pandas as pd
import altair as alt

from itertools import product
from vega_datasets import data

import warnings
warnings.filterwarnings('ignore')

In [53]:
# from google.colab import files
# uploaded = files.upload()

In [54]:
mass_shootings = pd.read_csv('MassShootings.csv')
county_population = pd.read_csv('CountyPopulation.csv')

**General Data Preparation**

In [55]:
mass_shootings['Incident Date'] = pd.to_datetime(mass_shootings['Incident Date'])
mass_shootings['Month_Year'] = mass_shootings['Incident Date'].dt.to_period('M').dt.to_timestamp()
mass_shootings['Year'] = mass_shootings['Month_Year'].apply(lambda x: x.year).astype(int)
mass_shootings = mass_shootings.drop('Incident Date', axis=1)

# grouping BY STATE AND MONTH
mass_shootings_states = mass_shootings.groupby(['FIPS', 'State', 'Month_Year', 'Year', 'Region', 'Population']).size().reset_index(name='Total Shootings')

# grouping BY REGION AND MONTH
mass_shootings_regions = mass_shootings.groupby(['FIPS', 'Region', 'Month_Year', 'Year']).size().reset_index(name='Total Shootings')
region_population = mass_shootings_states.drop_duplicates('State').groupby(['Region'])['Population'].sum()
mass_shootings_regions = mass_shootings_regions.merge(region_population, on='Region')

## **First question**
#### **How has the number of mass shootings evolved in the big US regions between two concrete years? And by States?**

**Additional data preparation**

In [56]:
mass_shootings_regions = mass_shootings_regions.groupby(['Region', 'Month_Year', 'Year', 'Population'])['Total Shootings'].sum().reset_index()
mass_shootings_states = mass_shootings_states.groupby(['State', 'Region', 'Month_Year', 'Year','FIPS'])['Total Shootings'].sum().reset_index()

dates = pd.date_range(start = '2014-01', end = '2023-12', freq = 'MS')
states = mass_shootings_states['State'].unique()
regions_states = mass_shootings_states[['Region', 'State']].drop_duplicates()

# crossing both lists to obtain a continuous dataframe with all dates-state pairs
date_state_combinations = list(product(dates, states))

cont_mass_shootings_states = pd.DataFrame(date_state_combinations, columns = ['Month_Year', 'State'])
cont_mass_shootings_states['Month_Year'] = cont_mass_shootings_states['Month_Year'].dt.to_period('M').dt.to_timestamp()
cont_mass_shootings_states['Shootings_Base'] = 0
cont_mass_shootings_states = cont_mass_shootings_states.merge(regions_states, on = ['State'], how = 'inner')

mass_shootings_states = cont_mass_shootings_states.merge(mass_shootings_states, on = ['Month_Year', 'State', 'Region'], how = 'left')
mass_shootings_states['Total Shootings'] = mass_shootings_states['Total Shootings'].fillna(mass_shootings_states['Shootings_Base'])
mass_shootings_states = mass_shootings_states.drop('Shootings_Base', axis=1)
mass_shootings_states['FIPS'] = pd.to_numeric(mass_shootings_states['FIPS'], errors='coerce')


state_selection = alt.selection_point(fields = ['State'])
region_selection = alt.selection_point(fields = ['Region'])
date_selection = alt.selection_interval(encodings=['x'])

color_palette = ['#E69F00', '#56B4E9', '#009E73', '#0072B2', '#CC79A7']
region_order = sorted(mass_shootings_regions['Region'].unique())
color_region = alt.condition(region_selection, alt.Color('Region:N', scale = alt.Scale(range = color_palette, domain = region_order), legend = None), alt.value('lightgray'))

## **Second question**
#### **Given a concrete year, how has the number of mass shooting per citizen grown or decreased across the different regions in the US compared to the first year of sampled data?**

**Additional data preparation**

In [57]:
mass_shootings_regions = mass_shootings_regions.groupby(['Region', 'Year', 'Population'])['Total Shootings'].sum().reset_index()
mass_shootings_regions['Year'] = mass_shootings_regions['Year'].astype(int)

# defining the proportion by population
mass_shootings_regions['Shootings per 10M citizens'] = mass_shootings_regions['Total Shootings'] / mass_shootings_regions['Population'] * 10**7
mass_shootings_regions = mass_shootings_regions.drop(['Population', 'Total Shootings'], axis=1)

# for the sake of correct slope chart plotting
mass_shootings_regions['Comparison'] = mass_shootings_regions['Year'].apply(lambda x: '2014' if x == 2014 else 'Comparison Year')

mass_shootings_2014 = mass_shootings_regions[mass_shootings_regions['Year'] == 2014].drop(['Year', 'Comparison'], axis=1)
mass_shootings_regions = mass_shootings_regions[mass_shootings_regions['Year'] != 2014]

for region in mass_shootings_regions['Region'].unique():
    for year in mass_shootings_regions['Year'].unique():
      new_row = pd.DataFrame({
          'Region': [region],
          'Year': [year],
          'Comparison': ['2014'],
          'Shootings per 10M citizens': [mass_shootings_2014[mass_shootings_2014['Region'] == region]['Shootings per 10M citizens'].iloc[0]]
      })

      mass_shootings_regions = pd.concat([mass_shootings_regions, new_row], ignore_index=True)

# separating the dataset by regions for posterior plot juxtaposition
mass_shootings_midwest = mass_shootings_regions[mass_shootings_regions['Region'] == 'Midwest']
mass_shootings_northeast = mass_shootings_regions[mass_shootings_regions['Region'] == 'Northeast']
mass_shootings_southeast = mass_shootings_regions[mass_shootings_regions['Region'] == 'Southeast']
mass_shootings_southwest = mass_shootings_regions[mass_shootings_regions['Region'] == 'Southwest']
mass_shootings_west = mass_shootings_regions[mass_shootings_regions['Region'] == 'West']

**Slopechart plotting**

In [58]:
select_year = alt.selection_point(encodings = ['color'])

color = alt.condition(select_year,
                      alt.Color('Year:N', legend = None, scale = alt.Scale(scheme='category10')),
                      alt.value('rgba(169, 169, 169, 0.3)')) # different color and lower opacity

slopecharts_regions = list()
region_dfs = [mass_shootings_midwest, mass_shootings_northeast, mass_shootings_southeast, mass_shootings_southwest, mass_shootings_west]
region_names = ['Midwest', 'Northeast', 'Southeast', 'Southwest', 'West']

for i in range(len(region_dfs)):
    df = region_dfs[i]
    region = region_names[i]

    slopechart = alt.Chart(df).mark_line(point = True).encode(
        x = alt.X('Comparison:N',
                  title = 'Time',
                  axis = alt.Axis(labelAngle = 45)), # rotation for better readibility
        y = alt.Y('Shootings per 10M citizens:Q',
              scale = alt.Scale(domain = [4,30]),
              title = 'Shootings per 10M citizens'),
        color = color,
        tooltip = 'Shootings per 10M citizens:Q'
    ).properties(title = alt.TitleParams(
        text = f'{region}',
        fontSize = 15,
        color = 'black',
        fontWeight='bold'),
                 width = 150,
                 height = 400
    ).add_params(select_year)

    slopecharts_regions.append(slopechart)

legend = alt.Chart(mass_shootings_regions).mark_circle(size = 70).encode(
    alt.Y('Year:N').axis(orient='right'),
    color = color,
).add_params(select_year)

Q2_slopecharts = alt.hconcat(*slopecharts_regions)
Q2_slopecharts_final = Q2_slopecharts | legend
Q2_slopecharts_final

alt.HConcatChart(...)

For this question, the main goal of the visualization is to aid the user to **identify trends** for certain years and by regions. Because the static comparison of each year selection only consists of **two temporal data points** we have displayed the information with **dotplots.**
To further offer insights on the trends by temporal encodings, we have displayed the **5 dotplots, one for each region, in a juxtaposed layout,** making it easier to analyse how trends appear for each one. By choosing this layout type over an overlay, we sacrifice a slight tradeoff in space-efficiency for **speed in task completion.**

The interactivity of this chart is based on the year choice that actively serves as the comparison point for the reference, 2014. By **selecting a year from the legend on the right,** only its color encoding is mantained in the dotplots, while the color for the rest of the lines becomes a light gray with lower opacity to make the selected one stand out over the others.

## **Third question**
#### **For the visualization in Q1, it should be possible to select a state, and show the detailed information on its counties.**

**Additional data preparation**

In [59]:
mass_shootings_states = mass_shootings_states.groupby(['FIPS', 'State'])['Total Shootings'].sum().reset_index()

missing_counties = pd.DataFrame([
    {'County FIPS': 2201, 'County Name': 'Prince of Wales-Outer Ketchikan, AK', 'County Population': 5696},
    {'County FIPS': 2232, 'County Name': 'Skagway-Hoonah-Angoon, AK', 'County Population': 2262},
    {'County FIPS': 2261, 'County Name': 'Valdez-Cordova, AK', 'County Population': 9202},
    {'County FIPS': 2270, 'County Name': 'Wade Hampton, AK', 'County Population': 8001},
    {'County FIPS': 2280, 'County Name': 'Wrangell-Petersburg, AK', 'County Population': 2064},
    {'County FIPS': 46113, 'County Name': 'Shannon County, SD', 'County Population': 13672},
    {'County FIPS': 51515, 'County Name': 'Bedford, VA', 'County Population': 6777},
])
county_population = pd.concat([county_population, missing_counties], ignore_index=True)
county_population = county_population[county_population['County FIPS']%1000 != 0] # erasing State FIPS

**Choropleth and map plotting**

In [62]:
USA_counties = alt.topo_feature(data.us_10m.url, 'counties')
USA_states = alt.topo_feature(data.us_10m.url, 'states')

domain = [2000, 100000, 1000000, 5000000, 10000000]
color_range = ['#e0e0e0', '#b3b3b3', '#808080', '#4d4d4d', '#2d2d2d']
slider = alt.binding_range(min = 2014, max = 2023, step = 1, name = 'Select a year: ')
year_selection = alt.param(bind = slider, value = 2014)
state_selection = alt.selection_point(fields = ['State'])

state_shape_overlay = alt.Chart(USA_states).transform_lookup(
      lookup = 'id',
      from_ = alt.LookupData(mass_shootings, 'FIPS', list(mass_shootings.columns))
  ).mark_geoshape(
      stroke = 'black',
      fill = 'transparent'
  ).encode(tooltip = ['State:N']).project(type = 'albersUsa').properties(
      width = 580,
      height = 500
  )

Q3_county_population_map = alt.Chart(USA_counties).transform_lookup(
      lookup = 'id',
      from_ = alt.LookupData(county_population, 'County FIPS', list(county_population.columns))
  ).mark_geoshape(
      stroke = 'darkgray',
      strokeWidth = 0.5,
      opacity = 0.7
  ).encode(
      color = alt.Color(
          'County Population:Q',
          legend = alt.Legend(
              title = 'County Population',
              titleColor = 'black',
              labelColor = 'black',
              labelLimit = 500
          ),
          scale=alt.Scale(domain=domain, range=color_range),
      ),
      tooltip = ['County Name:N', 'County Population:Q']
  ).project(type = 'albersUsa').properties(
      width = 580,
      height = 500
  )

Q3_county_shootings = alt.Chart(mass_shootings).mark_circle().encode(
      longitude = 'Longitude:Q',
      latitude = 'Latitude:Q',
      size = alt.value(12),
      color = alt.value('#003E5C'),
      opacity = alt.condition(alt.datum.Year == year_selection, alt.value(1), alt.value(0))
  ).project(type = 'albersUsa').properties(
      width=580,
      height=500
  ).add_params(year_selection)

selected_state_overlay = alt.Chart(USA_states).transform_lookup(
      lookup = 'id',
      from_ = alt.LookupData(mass_shootings_states, 'FIPS', list(mass_shootings_states.columns))
  ).mark_geoshape(strokeWidth = 0).encode(
      color = alt.value('white'),
      opacity = alt.condition(state_selection, alt.value(0), alt.value(1)),
      tooltip = ['State:N']
  ).project(type = 'albersUsa').properties(
      title = alt.TitleParams(
            text = 'Distribution of shootings per year, by state and county',
            fontSize = 18,
            fontWeight = 'bold',
            color = 'black'),
        width = 580,
        height = 500
  ).add_params(
      state_selection
  )

selected_county_overlay = alt.Chart(USA_counties
  ).transform_lookup(
      lookup = 'id',
      from_ = alt.LookupData(county_population, 'County FIPS', list(county_population.columns))
  ).mark_geoshape(
      fill = 'transparent'
  ).encode(
      tooltip = ['County Name:N']
  ).project(
      type = 'albersUsa'
  ).properties(
      width=580,
      height=500
  )

Q3_final_chart = (state_shape_overlay + Q3_county_population_map + Q3_county_shootings + selected_state_overlay + selected_county_overlay)
Q3_final_chart

alt.LayerChart(...)

For the third question, the selection of the finally displayed information was crucial, so that is why we opted to **combine two visualizations** to answer it.

Firstly, we represented the spots where mass shootings happened, with **coordinates in a United States map.** In order to further analyze the absolute quantities that were being represented, we used a **choropleth as a base, encoding county population with a grayscale quantitative palette** in a fitted domain to ensure easier value distinction.

Moreover, we used a **line chart with 3 lines to display the evolution of the top 3 counties per each state,** that conveniently **appears when said state is selected** dynamically in the left-side chart. That way, knowing the amount of counties per state is undistinguishable at a first glance, we are able to just display the full evolution of the most significant cases for each one, giving more valuable insights.

The selection of the state is not the only interactive element in these charts, as this visualization is connected to the one used for question one and so **the time range can also be dinamically selected.** The chosen domain encodes the x-axis range of the line chart appearing on the right, as well as the amount of points represented in the left-side chart.